###### Problem 1: PSO

In [632]:
import numpy as np
import itertools
import math

In [633]:
np.random.seed(2021)

In [634]:
def sphere(x, dim):
    return sum(_x ** 2 for _x in x[:dim])
def rastrigin(x, dim):
    return 10 * dim + sum(_x ** 2-10 * math.cos(2 * math.pi * _x) for _x in x[:dim])

In [635]:
class Particle: # all the material that is relavant at the level of the individual particles
    
    def __init__(self, dim, minx, maxx, func):
#         print(dim,minx,maxx)
        self.position = np.random.uniform(low=minx, high=maxx, size=dim)# Position of a particle
        self.velocity = np.random.uniform(low=-0.1, high=0.1, size=dim) # velocity of a particle
        self.best_particle_pos = self.position # local optima
        self.dim = dim
        
        self.fit_func = func
        self.fitness = self.fit_func(self.position,dim)
        self.best_particle_fitness = self.fitness   # we couldd start with very large number here, 
                                                    #but the actual value is better in case we are lucky 
                
    def setPos(self, pos):
        self.position = pos
        self.fitness = self.fit_func(self.position,self.dim)
        if self.fitness<self.best_particle_fitness:     # to update the personal best both 
                                                        # position (for velocity update) and
                                                        # fitness (the new standard) are needed
                                                        # global best is update on swarm leven
            self.best_particle_fitness = self.fitness
            self.best_particle_pos = pos

    def updateVel(self, inertia, a1, a2, best_self_pos, best_swarm_pos):
                # Here we use the canonical version
                # V <- inertia*V + a1r1 (peronal_best - current_pos) + a2r2 (global_best - current_pos)
        cur_vel = self.velocity
        r1 = np.random.uniform(low=0, high=1, size = self.dim) # randomize
        r2 = np.random.uniform(low=0, high=1, size = self.dim)
        a1r1 = np.multiply(a1, r1)
        a2r2 = np.multiply(a2, r2)
        best_self_dif = np.subtract(best_self_pos, self.position)
        best_swarm_dif = np.subtract(best_swarm_pos, self.position)
                    # the next line is the main equation, namely the velocity update, 
                    # the velocities are added to the positions at swarm level
        new_vel = inertia*cur_vel + np.multiply(a1r1, best_self_dif) + np.multiply(a2r2, best_swarm_dif)
        self.velocity = new_vel
        return new_vel

In [636]:
class PSO: # all the material that is relavant at swarm leveel

    def __init__(self, w, a1, a2, dim, population_size, time_steps, search_range, func):

        self.w = w # Inertia
        self.a1 = a1 # Attraction to personal best
        self.a2 = a2 # Attraction to global best
        self.dim = dim
        self.fitness_func = func
        self.swarm = [Particle(dim,-search_range,search_range, func) for i in range(population_size)]
        self.time_steps = time_steps
#         print('init')

        # Initialising global best, you can wait until the end of the first time step
        # but creating a random initial best and fitness which is very high will mean you
        # do not have to write an if statement for the one off case
        self.best_swarm_pos = np.random.uniform(low=-500, high=500, size=dim)
        self.best_swarm_fitness = 1e100

    def particle_newpos(self, p, t): # return the new position of a particle
        particle = self.swarm[p]
        new_position = particle.position + particle.updateVel(self.w, self.a1, self.a2, particle.best_particle_pos, self.best_swarm_pos)
        return new_position
    def run(self):
        convergence_count = 0
        for t in range(self.time_steps):
#             print('t:',t)
            for p in range(len(self.swarm)):
                new_position = self.particle_newpos(p, t) # t is the time step now
                                
                if new_position@new_position > 1.0e+18: # The search will be terminated if the distance 
                                                        # of any particle from center is too large
                    print('Time:', t,'Best Pos:',self.best_swarm_pos,'Best Fit:',self.best_swarm_fitness)
#                     raise SystemExit('Most likely divergent: Decrease parameter values')
                    return [self.best_swarm_fitness,np.inf,False]
                self.swarm[p].setPos(new_position)

                new_fitness = self.fitness_func(new_position,self.dim)

                if new_fitness < self.best_swarm_fitness:   # to update the global best both 
                                                            # position (for velocity update) and
                                                            # fitness (the new group norm) are needed
                    self.best_swarm_fitness = new_fitness
                    self.best_swarm_pos = new_position
                
#             if t % 100 == 0: #we print only two components even it search space is high-dimensional
#                 print("Time: %6d,  Best Fitness: %14.6f,  Best Pos: %9.4f,%9.4f" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1]), end =" ")
#                 if self.dim>2: 
#                     print('...')
#                 else:
#                     print('')
            if self.best_swarm_fitness<1e-7:
                convergence_count+=1
                if convergence_count>=10:
#                     print("PSO Converged at iteration %d"%t)
                    return [self.best_swarm_fitness,t,True]
        return [self.best_swarm_fitness,t,False]

In [392]:
a = 2.0
result = PSO(dim=50, w=0.82, a1=a/2, a2=a/2, population_size=20, time_steps=5001, search_range=5.12, func=sphere).run()
print('Converge = ',result[2],', at time = ', result[1], ' |  best_fitness = ',result[0])

Converge =  False , at time =  2000  |  best_fitness =  1.1780949271483111e-05


In [637]:
#Script
def test(f,dim,population,itr):
    
    d = [('Converge',bool),('Time','float'),('Dim','int'),('w','float'),('a1','float'),('a2','float'),('population_size','int'),('time_step','int'),('search_range','float')]
    values =np.zeros(itr,dtype=d)
    for i in range(itr):
        a = np.random.choice(np.arange(2.0,2.5,0.1)) # [2.0, 2.4]
#         dim = np.random.choice(np.arange(5,51,5)) # [5, 50]
        dim = dim
        w = round( np.random.choice(np.arange(0.80,0.84,0.01)), 2) # [0.80, 0.83]
        a1 = round(a/2 + np.random.choice(np.arange(-0.1,0.15,0.05)) ,1)# [0.9, 1.3]
        a2 = round(a - a1,1)
#         population_size = np.random.choice(np.arange(20,50,10)) # [20, 40]
        population_size = population
        search_range = 5.12
        time_steps = 5001
        function = str(f.__name__)
#         print(type(func.__name__))
        result = PSO(w, a1, a2,dim, population_size, time_steps, search_range, func=f).run()
#         print('Iteration = ',i,' dim=',dim,', w=',w,', a1 =',a1,', a2=',a2,', population_size=', population_size)
        v = np.array([(result[2],result[1],dim,w,a1,a2,population_size,time_steps,search_range)],dtype=d)
#         print(function)
#         print(v)
#         values = np.append(values,v)
        values[i] = v
#     print(values)
    values = np.sort(np.array(values),order = 'Time')
#     print(function," function sorted result:\n",values)
    return values



In [406]:
output_sphere = test(sphere,50,30,50)
# print(sphere.__name__)

sphere  function sorted result:
 [( True,  868., 50, 0.81, 1.3, 1.1, 30, 5001, 5.12)
 ( True,  875., 50, 0.82, 1.2, 1. , 30, 5001, 5.12)
 ( True,  896., 50, 0.8 , 1.3, 1.1, 30, 5001, 5.12)
 ( True,  920., 50, 0.82, 1. , 1. , 30, 5001, 5.12)
 ( True,  934., 50, 0.8 , 1.1, 1. , 30, 5001, 5.12)
 ( True,  971., 50, 0.81, 1.2, 1. , 30, 5001, 5.12)
 ( True, 1025., 50, 0.82, 1. , 1.1, 30, 5001, 5.12)
 ( True, 1034., 50, 0.83, 1.1, 0.9, 30, 5001, 5.12)
 ( True, 1036., 50, 0.81, 1.2, 1.2, 30, 5001, 5.12)
 ( True, 1039., 50, 0.8 , 1.2, 1. , 30, 5001, 5.12)
 ( True, 1080., 50, 0.8 , 1. , 1. , 30, 5001, 5.12)
 ( True, 1144., 50, 0.8 , 1. , 1.2, 30, 5001, 5.12)
 ( True, 1153., 50, 0.8 , 1. , 1.2, 30, 5001, 5.12)
 ( True, 1163., 50, 0.8 , 1.2, 1.1, 30, 5001, 5.12)
 ( True, 1164., 50, 0.82, 1.2, 1. , 30, 5001, 5.12)
 ( True, 1192., 50, 0.81, 1.3, 1.1, 30, 5001, 5.12)
 ( True, 1224., 50, 0.8 , 1. , 1. , 30, 5001, 5.12)
 ( True, 1226., 50, 0.82, 1.2, 1. , 30, 5001, 5.12)
 ( True, 1229., 50, 0.8 , 1. , 

In [407]:
import sys
from prettytable import PrettyTable # You need to type in command: pip install prettytable
PYTHONIOENCODING="UTF-8"  

table_sphere = PrettyTable(['Converge', 'Converge Time','Dim','w','a1','a2','Population','Time_step','Range'])
[table_sphere.add_row(output_sphere[i]) for i in range(output_sphere.shape[0])]
print(table_sphere)

+----------+---------------+-----+------+-----+-----+------------+-----------+-------+
| Converge | Converge Time | Dim |  w   |  a1 |  a2 | Population | Time_step | Range |
+----------+---------------+-----+------+-----+-----+------------+-----------+-------+
|   True   |     868.0     |  50 | 0.81 | 1.3 | 1.1 |     30     |    5001   |  5.12 |
|   True   |     875.0     |  50 | 0.82 | 1.2 | 1.0 |     30     |    5001   |  5.12 |
|   True   |     896.0     |  50 | 0.8  | 1.3 | 1.1 |     30     |    5001   |  5.12 |
|   True   |     920.0     |  50 | 0.82 | 1.0 | 1.0 |     30     |    5001   |  5.12 |
|   True   |     934.0     |  50 | 0.8  | 1.1 | 1.0 |     30     |    5001   |  5.12 |
|   True   |     971.0     |  50 | 0.81 | 1.2 | 1.0 |     30     |    5001   |  5.12 |
|   True   |     1025.0    |  50 | 0.82 | 1.0 | 1.1 |     30     |    5001   |  5.12 |
|   True   |     1034.0    |  50 | 0.83 | 1.1 | 0.9 |     30     |    5001   |  5.12 |
|   True   |     1036.0    |  50 | 0.81 | 1

In [457]:
def test1_2_ras():   
    data_ras = PrettyTable(['Dim','Population','Average Converge Time','Converge Rate'])
    for i in range(3,5):
        for j in range (40,161,40):
            o_p = test(rastrigin,i,j,itr=50)  
            dims = i
            pops = j
            avConTime = round(o_p['Time'][o_p['Converge']].mean(),2)
            ConRate = o_p['Converge'].sum()/50
            data_ras.add_row(np.array([dims,pops,avConTime,ConRate]))
    print(data_ras)
test1_2_ras()

rastrigin  function sorted result:
 [( True,  113., 3, 0.8 , 1.2, 0.9, 40, 5001, 5.12)
 ( True,  117., 3, 0.8 , 1.3, 1. , 40, 5001, 5.12)
 ( True,  123., 3, 0.8 , 1. , 1.1, 40, 5001, 5.12)
 ( True,  123., 3, 0.81, 1.1, 0.9, 40, 5001, 5.12)
 ( True,  127., 3, 0.8 , 1.3, 1.1, 40, 5001, 5.12)
 ( True,  132., 3, 0.8 , 1.2, 0.9, 40, 5001, 5.12)
 ( True,  137., 3, 0.8 , 1. , 1.1, 40, 5001, 5.12)
 ( True,  137., 3, 0.81, 0.9, 1.1, 40, 5001, 5.12)
 ( True,  139., 3, 0.8 , 1.3, 1.1, 40, 5001, 5.12)
 ( True,  142., 3, 0.8 , 1. , 1.3, 40, 5001, 5.12)
 ( True,  144., 3, 0.8 , 1.3, 1.1, 40, 5001, 5.12)
 ( True,  144., 3, 0.82, 1. , 1.2, 40, 5001, 5.12)
 ( True,  146., 3, 0.8 , 1. , 1.1, 40, 5001, 5.12)
 ( True,  147., 3, 0.8 , 1.2, 1.1, 40, 5001, 5.12)
 ( True,  152., 3, 0.8 , 1. , 1.1, 40, 5001, 5.12)
 ( True,  152., 3, 0.81, 1.1, 1. , 40, 5001, 5.12)
 ( True,  153., 3, 0.81, 1.3, 1.1, 40, 5001, 5.12)
 ( True,  156., 3, 0.8 , 1.1, 1.3, 40, 5001, 5.12)
 ( True,  158., 3, 0.81, 1.2, 1. , 40, 5001, 5

rastrigin  function sorted result:
 [( True,  114., 4, 0.81, 1. , 1. , 40, 5001, 5.12)
 ( True,  159., 4, 0.8 , 1.1, 1.2, 40, 5001, 5.12)
 ( True,  162., 4, 0.81, 1.1, 1. , 40, 5001, 5.12)
 ( True,  162., 4, 0.81, 1.2, 0.9, 40, 5001, 5.12)
 ( True,  168., 4, 0.8 , 1. , 1.1, 40, 5001, 5.12)
 ( True,  168., 4, 0.8 , 1.2, 1. , 40, 5001, 5.12)
 ( True,  169., 4, 0.8 , 1.1, 1. , 40, 5001, 5.12)
 ( True,  172., 4, 0.8 , 1.1, 0.9, 40, 5001, 5.12)
 ( True,  179., 4, 0.83, 1.2, 0.9, 40, 5001, 5.12)
 ( True,  180., 4, 0.81, 1. , 1.3, 40, 5001, 5.12)
 ( True,  182., 4, 0.8 , 1.1, 1.1, 40, 5001, 5.12)
 ( True,  183., 4, 0.81, 1.2, 0.9, 40, 5001, 5.12)
 ( True,  186., 4, 0.81, 1.1, 1.2, 40, 5001, 5.12)
 ( True,  191., 4, 0.82, 1.2, 1.1, 40, 5001, 5.12)
 ( True,  196., 4, 0.8 , 1. , 1. , 40, 5001, 5.12)
 ( True,  203., 4, 0.8 , 1. , 1.1, 40, 5001, 5.12)
 ( True,  209., 4, 0.82, 1. , 1.2, 40, 5001, 5.12)
 ( True,  213., 4, 0.83, 1. , 1. , 40, 5001, 5.12)
 ( True,  220., 4, 0.82, 1.1, 0.9, 40, 5001, 5

In [458]:
def test1_2_sph():
    data_sph = PrettyTable(['Dim','Population','Average Converge Time','Converge Rate'])
    for i in range(30,71,10):
        for j in range (20,51,10):
            o_p = test(sphere,i,j,itr=50)  
            dims = i
            pops = j
            avConTime = round(o_p['Time'][o_p['Converge']].mean(),2)
            ConRate = o_p['Converge'].sum()/50
            data_sph.add_row(np.array([dims,pops,avConTime,ConRate]))
    print(data_sph)
test1_2_sph()

sphere  function sorted result:
 [( True,  498., 30, 0.81, 1.3, 1. , 20, 5001, 5.12)
 ( True,  500., 30, 0.81, 1. , 1. , 20, 5001, 5.12)
 ( True,  504., 30, 0.8 , 1.2, 1. , 20, 5001, 5.12)
 ( True,  559., 30, 0.82, 1. , 1. , 20, 5001, 5.12)
 ( True,  560., 30, 0.8 , 1.2, 0.9, 20, 5001, 5.12)
 ( True,  566., 30, 0.8 , 1.3, 1. , 20, 5001, 5.12)
 ( True,  588., 30, 0.8 , 1. , 1. , 20, 5001, 5.12)
 ( True,  590., 30, 0.83, 1.2, 0.9, 20, 5001, 5.12)
 ( True,  601., 30, 0.8 , 1. , 1. , 20, 5001, 5.12)
 ( True,  621., 30, 0.82, 1.2, 1.2, 20, 5001, 5.12)
 ( True,  627., 30, 0.82, 1.1, 1. , 20, 5001, 5.12)
 ( True,  629., 30, 0.83, 1.3, 1. , 20, 5001, 5.12)
 ( True,  647., 30, 0.8 , 1. , 1. , 20, 5001, 5.12)
 ( True,  650., 30, 0.82, 1.3, 1. , 20, 5001, 5.12)
 ( True,  653., 30, 0.83, 1.2, 1. , 20, 5001, 5.12)
 ( True,  658., 30, 0.81, 1. , 1. , 20, 5001, 5.12)
 ( True,  669., 30, 0.83, 1.1, 1.2, 20, 5001, 5.12)
 ( True,  678., 30, 0.83, 1.2, 1.1, 20, 5001, 5.12)
 ( True,  685., 30, 0.83, 1.2, 

sphere  function sorted result:
 [( True,  881., 40, 0.81, 1.3, 1.1, 20, 5001, 5.12)
 ( True,  948., 40, 0.81, 1.1, 0.9, 20, 5001, 5.12)
 ( True,  972., 40, 0.83, 1.2, 1. , 20, 5001, 5.12)
 ( True,  984., 40, 0.81, 1.2, 1.2, 20, 5001, 5.12)
 ( True, 1099., 40, 0.8 , 1. , 1.1, 20, 5001, 5.12)
 ( True, 1113., 40, 0.8 , 1.2, 1.1, 20, 5001, 5.12)
 ( True, 1159., 40, 0.8 , 1.2, 1. , 20, 5001, 5.12)
 ( True, 1166., 40, 0.8 , 1.1, 1. , 20, 5001, 5.12)
 ( True, 1181., 40, 0.83, 1.3, 1.1, 20, 5001, 5.12)
 ( True, 1182., 40, 0.81, 1.2, 1. , 20, 5001, 5.12)
 ( True, 1191., 40, 0.82, 1. , 1.2, 20, 5001, 5.12)
 ( True, 1196., 40, 0.81, 1. , 1.1, 20, 5001, 5.12)
 ( True, 1227., 40, 0.81, 1.2, 1.1, 20, 5001, 5.12)
 ( True, 1234., 40, 0.8 , 1. , 1.1, 20, 5001, 5.12)
 ( True, 1236., 40, 0.82, 1.2, 1. , 20, 5001, 5.12)
 ( True, 1290., 40, 0.8 , 1.1, 0.9, 20, 5001, 5.12)
 ( True, 1314., 40, 0.81, 1. , 1.2, 20, 5001, 5.12)
 ( True, 1317., 40, 0.82, 1. , 1.2, 20, 5001, 5.12)
 ( True, 1339., 40, 0.82, 1. , 

sphere  function sorted result:
 [( True, 1603., 50, 0.83, 1.3, 1. , 20, 5001, 5.12)
 ( True, 1679., 50, 0.82, 1.2, 1.1, 20, 5001, 5.12)
 ( True, 1808., 50, 0.8 , 1.3, 1.1, 20, 5001, 5.12)
 ( True, 1835., 50, 0.81, 1.3, 1.1, 20, 5001, 5.12)
 ( True, 1900., 50, 0.83, 1. , 1. , 20, 5001, 5.12)
 ( True, 1902., 50, 0.81, 1.2, 1.1, 20, 5001, 5.12)
 ( True, 1920., 50, 0.8 , 1.3, 1. , 20, 5001, 5.12)
 ( True, 1922., 50, 0.81, 1. , 1.1, 20, 5001, 5.12)
 ( True, 1932., 50, 0.81, 1. , 1.1, 20, 5001, 5.12)
 ( True, 1933., 50, 0.8 , 1. , 1.3, 20, 5001, 5.12)
 ( True, 1946., 50, 0.8 , 1.1, 1.1, 20, 5001, 5.12)
 ( True, 1981., 50, 0.8 , 1. , 1.2, 20, 5001, 5.12)
 ( True, 2024., 50, 0.82, 1.2, 1.2, 20, 5001, 5.12)
 ( True, 2078., 50, 0.82, 1.1, 0.9, 20, 5001, 5.12)
 ( True, 2087., 50, 0.81, 1. , 1.1, 20, 5001, 5.12)
 ( True, 2120., 50, 0.82, 1.1, 1. , 20, 5001, 5.12)
 ( True, 2123., 50, 0.8 , 1.1, 1.3, 20, 5001, 5.12)
 ( True, 2174., 50, 0.82, 1.2, 1.2, 20, 5001, 5.12)
 ( True, 2215., 50, 0.83, 1.3, 

sphere  function sorted result:
 [( True, 2242., 60, 0.8 , 1.2, 1.1, 20, 5001, 5.12)
 ( True, 2444., 60, 0.8 , 1.2, 1. , 20, 5001, 5.12)
 ( True, 2462., 60, 0.8 , 1.2, 1.2, 20, 5001, 5.12)
 ( True, 2490., 60, 0.8 , 1.3, 1.1, 20, 5001, 5.12)
 ( True, 2547., 60, 0.8 , 1.2, 1.2, 20, 5001, 5.12)
 ( True, 2639., 60, 0.83, 1.2, 1.1, 20, 5001, 5.12)
 ( True, 2662., 60, 0.8 , 1.2, 1. , 20, 5001, 5.12)
 ( True, 2670., 60, 0.83, 1.2, 1. , 20, 5001, 5.12)
 ( True, 2744., 60, 0.82, 1.2, 1. , 20, 5001, 5.12)
 ( True, 2862., 60, 0.8 , 1.2, 1. , 20, 5001, 5.12)
 ( True, 2876., 60, 0.8 , 1.2, 0.9, 20, 5001, 5.12)
 ( True, 2976., 60, 0.83, 1.2, 1. , 20, 5001, 5.12)
 ( True, 3028., 60, 0.82, 1. , 1.3, 20, 5001, 5.12)
 ( True, 3044., 60, 0.83, 1.2, 1. , 20, 5001, 5.12)
 ( True, 3053., 60, 0.8 , 1.3, 1.1, 20, 5001, 5.12)
 ( True, 3057., 60, 0.8 , 1.3, 1.1, 20, 5001, 5.12)
 ( True, 3112., 60, 0.82, 1. , 1.3, 20, 5001, 5.12)
 ( True, 3129., 60, 0.83, 1.1, 1.2, 20, 5001, 5.12)
 ( True, 3162., 60, 0.83, 1.2, 

sphere  function sorted result:
 [( True, 3865., 70, 0.8 , 1.3, 1.1, 20, 5001, 5.12)
 ( True, 4008., 70, 0.81, 1.3, 1.1, 20, 5001, 5.12)
 ( True, 4097., 70, 0.82, 1.1, 1.3, 20, 5001, 5.12)
 ( True, 4217., 70, 0.82, 1.1, 1.1, 20, 5001, 5.12)
 ( True, 4620., 70, 0.83, 1.2, 1.2, 20, 5001, 5.12)
 ( True, 4745., 70, 0.81, 1.2, 1.1, 20, 5001, 5.12)
 ( True, 4763., 70, 0.83, 1.1, 1.3, 20, 5001, 5.12)
 ( True, 4775., 70, 0.83, 1.3, 1. , 20, 5001, 5.12)
 ( True, 4985., 70, 0.8 , 1. , 1. , 20, 5001, 5.12)
 (False, 5000., 70, 0.8 , 1. , 1. , 20, 5001, 5.12)
 (False, 5000., 70, 0.8 , 1. , 1. , 20, 5001, 5.12)
 (False, 5000., 70, 0.8 , 1. , 1. , 20, 5001, 5.12)
 (False, 5000., 70, 0.8 , 1. , 1. , 20, 5001, 5.12)
 (False, 5000., 70, 0.8 , 1. , 1. , 20, 5001, 5.12)
 (False, 5000., 70, 0.8 , 1. , 1.1, 20, 5001, 5.12)
 (False, 5000., 70, 0.8 , 1. , 1.1, 20, 5001, 5.12)
 (False, 5000., 70, 0.8 , 1. , 1.1, 20, 5001, 5.12)
 (False, 5000., 70, 0.8 , 1. , 1.2, 20, 5001, 5.12)
 (False, 5000., 70, 0.8 , 1.1, 

In [416]:
output_rastrigin = test(rastrigin,4,120,itr=50)

rastrigin  function sorted result:
 [( True, 120., 4, 0.8 , 1. , 1.1, 120, 5001, 5.12)
 ( True, 141., 4, 0.8 , 1.2, 0.9, 120, 5001, 5.12)
 ( True, 144., 4, 0.8 , 1. , 1.2, 120, 5001, 5.12)
 ( True, 149., 4, 0.81, 1. , 1. , 120, 5001, 5.12)
 ( True, 152., 4, 0.8 , 1.2, 1.2, 120, 5001, 5.12)
 ( True, 153., 4, 0.82, 1. , 1.1, 120, 5001, 5.12)
 ( True, 154., 4, 0.8 , 1. , 1. , 120, 5001, 5.12)
 ( True, 155., 4, 0.81, 1. , 1.1, 120, 5001, 5.12)
 ( True, 156., 4, 0.81, 1.2, 1. , 120, 5001, 5.12)
 ( True, 161., 4, 0.81, 1. , 1. , 120, 5001, 5.12)
 ( True, 161., 4, 0.81, 1. , 1.2, 120, 5001, 5.12)
 ( True, 166., 4, 0.81, 1. , 1.2, 120, 5001, 5.12)
 ( True, 168., 4, 0.81, 1.2, 0.9, 120, 5001, 5.12)
 ( True, 170., 4, 0.8 , 1. , 1. , 120, 5001, 5.12)
 ( True, 172., 4, 0.83, 1.1, 0.9, 120, 5001, 5.12)
 ( True, 176., 4, 0.81, 1. , 1. , 120, 5001, 5.12)
 ( True, 177., 4, 0.82, 1. , 1.2, 120, 5001, 5.12)
 ( True, 178., 4, 0.82, 1.2, 1.2, 120, 5001, 5.12)
 ( True, 178., 4, 0.83, 1.3, 1.1, 120, 5001, 5

In [420]:
table_rastrigin = PrettyTable(['Converge', 'Converge Time','Dim','w','a1','a2','Population','Time_step','Range'])
[table_rastrigin.add_row(output_rastrigin[i]) for i in range(output_rastrigin.shape[0])]
print(table_rastrigin)


+----------+---------------+-----+------+-----+-----+------------+-----------+-------+
| Converge | Converge Time | Dim |  w   |  a1 |  a2 | Population | Time_step | Range |
+----------+---------------+-----+------+-----+-----+------------+-----------+-------+
|   True   |     120.0     |  4  | 0.8  | 1.0 | 1.1 |    120     |    5001   |  5.12 |
|   True   |     141.0     |  4  | 0.8  | 1.2 | 0.9 |    120     |    5001   |  5.12 |
|   True   |     144.0     |  4  | 0.8  | 1.0 | 1.2 |    120     |    5001   |  5.12 |
|   True   |     149.0     |  4  | 0.81 | 1.0 | 1.0 |    120     |    5001   |  5.12 |
|   True   |     152.0     |  4  | 0.8  | 1.2 | 1.2 |    120     |    5001   |  5.12 |
|   True   |     153.0     |  4  | 0.82 | 1.0 | 1.1 |    120     |    5001   |  5.12 |
|   True   |     154.0     |  4  | 0.8  | 1.0 | 1.0 |    120     |    5001   |  5.12 |
|   True   |     155.0     |  4  | 0.81 | 1.0 | 1.1 |    120     |    5001   |  5.12 |
|   True   |     156.0     |  4  | 0.81 | 1

In [446]:
class ParticleDynamic():
    def __init__(self, dim, minx, maxx, func):
#         print(dim,minx,maxx)
        self.position = np.random.uniform(low=minx, high=maxx, size=dim)# Position of a particle
        self.velocity = np.random.uniform(low=-0.1, high=0.1, size=dim) # velocity of a particle
        self.best_particle_pos = self.position # local optima
        self.dim = dim
        
        self.fit_func = func
        self.fitness = self.fit_func(self.position,dim)
        self.best_particle_fitness = self.fitness   # we couldd start with very large number here, 
                                                    #but the actual value is better in case we are lucky 
                
    def setPos(self, pos):
        self.position = pos
        self.fitness = self.fit_func(self.position,self.dim)
        if self.fitness<self.best_particle_fitness:     # to update the personal best both 
                                                        # position (for velocity update) and
                                                        # fitness (the new standard) are needed
                                                        # global best is update on swarm leven
            self.best_particle_fitness = self.fitness
            self.best_particle_pos = pos
    def updateVel(self, inertia, a1, a2, swarm, best_self_pos, best_swarm_pos):
        cur_vel = self.velocity
        r1 = np.random.uniform(low=0, high=1, size = self.dim)
        r2 = np.random.uniform(low=0, high=1, size = self.dim)
        #r3 = np.random.uniform(low=0, high=1, size = self.dim)
        a1r1 = np.multiply(a1, r1)
        a2r2 = np.multiply(a2, r2)
        best_self_dif = np.subtract(best_self_pos, self.position)#local
        best_swarm_dif = np.subtract(best_swarm_pos, self.position)#global
                    # the next line is the main equation, namely the velocity update, 
                    # the velocities are added to the positions at swarm level
        new_vel = inertia*cur_vel + np.multiply(a1r1, best_self_dif) + np.multiply(a2r2, best_swarm_dif)
        # Optimization 2: Use contriction factor K to ensure convergence 
        fi = a1 + a2  
        K = 2 / abs(2-fi-math.sqrt(abs(fi**2-4*fi)))
#         print('K = ',K)
        new_vel *= K
        self.velocity = new_vel
        return new_vel

In [463]:
class PSODynamic():
    def __init__(self, w_e, a1, a2, dim, population_size, time_steps, search_range, func):
#         super(PSODynamic,self).__init__(w_e, a1, a2, dim, population_size, time_steps, search_range, func)
        self.a1 = a1 # Attraction to personal best
        self.a2 = a2 # Attraction to global best
        self.dim = dim
        self.fitness_func = func
        self.swarm = [Particle(dim,-search_range,search_range, func) for i in range(population_size)]
        self.time_steps = time_steps
#         self.w_s = w_s # dynamic w
        self.w_e = w_e
        self.swarm = [ParticleDynamic(dim,-search_range,search_range, func) for i in range(population_size)]
        self.best_swarm_pos = np.random.uniform(low=-500, high=500, size=dim)
        self.best_swarm_fitness = 1e100
    def particle_newpos(self, p, t, w_s):
        self.this_fitness = self.best_swarm_fitness
        particle = self.swarm[p]
        # Optimization 1: Dynamic inertia
        w = self.w_e + (w_s - self.w_e) * (self.time_steps - t) / self.time_steps
        new_position = particle.position + particle.updateVel(w, self.a1, self.a2, self.swarm, particle.best_particle_pos, self.best_swarm_pos)
        return new_position
    def run(self):
        convergence_count = 0
        w_s = 0.86
        for t in range(self.time_steps):
#             print('t:',t)
            for p in range(len(self.swarm)):
                
                new_position = self.particle_newpos(p, t,w_s) # t is the time step now
                # dynamically update w_s to current w_e for next position update
                w_s = self.w_e + (w_s - self.w_e) * (self.time_steps - t) / self.time_steps
#                 w_s = self.w_e + (w_s - self.w_e) * t / self.time_steps
                
                if new_position@new_position > 1.0e+18: # The search will be terminated if the distance 
                                                        # of any particle from center is too large
                    print('Time:', t,'Best Pos:',self.best_swarm_pos,'Best Fit:',self.best_swarm_fitness)
                    raise SystemExit('Most likely divergent: Decrease parameter values')
                
                self.swarm[p].setPos(new_position)

                new_fitness = self.fitness_func(new_position,self.dim)

                if new_fitness < self.best_swarm_fitness:   # to update the global best both 
                                                            # position (for velocity update) and
                                                            # fitness (the new group norm) are needed
                    self.best_swarm_fitness = new_fitness
                    self.best_swarm_pos = new_position
                
#             if t % 100 == 0: #we print only two components even it search space is high-dimensional
#                 print("Time: %6d,  Best Fitness: %14.6f,  Best Pos: %9.4f,%9.4f" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1]), end =" ")
#                 if self.dim>2: 
#                     print('...')
#                 else:
#                     print('')
            if self.best_swarm_fitness<1e-7:
                convergence_count+=1
                if convergence_count>=10:
#                     print("PSODynamic Converged at iteration %d"%t)
                    return [self.best_swarm_fitness,t,True]
        return [self.best_swarm_fitness,t,False]

In [449]:
a = 4.08
PSODynamic(dim=50, w_e =0.82,a1=a/2, a2=a/2, population_size=40, time_steps=2001, search_range=5.12, func=sphere).run()
PSO(dim=50, w = 0.82,a1=1.2, a2=0.9, population_size=40, time_steps=2001, search_range=5.12, func=sphere).run()

PSODynamic Converged at iteration 626


[6.839530519767611e-08, 854, True]

In [ ]:
# Code 5
def compare(f,dim,population,itr):
    result = 0
    t1 = []
    t2 = []
    for i in range(itr):
        a = 4.08
        o1= PSODynamic(dim=50, w_e =0.82,a1=a/2, a2=a/2, population_size=population, time_steps=5001, search_range=5.12, func=sphere).run()
        o2= PSO(dim=50, w = 0.82,a1=1.1, a2=1.1, population_size=population, time_steps=5001, search_range=5.12, func=sphere).run()
        if (o1[1]<o2[1]): 
            result += 1
            t1.append(o1[1])
            t2.append(o2[1])
    a1 = round(np.array(t1).mean(),2)
    a2 = round(np.array(t2).mean(),2)
    Dif = round(a2 - a1,2)
    print('Dynamic PSO algorithm win *',result,'* times/',itr,'times \n','Average Converge time for DynamicPSO is ',a1,'\n'
          ,'Average Converge time for PSO is ',a2,'\n','which is ', Dif , 'steps quicker!')
    

In [464]:
compare(sphere,50,40,50)

Dynamic PSO algorithm win * 49 * times/ 50 times 
 Average Converge time for DynamicPSO is  618.45 
 Average Converge time for PSO is  984.59 
 which is  366.14 steps quicker!


In [465]:
compare(rastrigin,4,120,50)

Dynamic PSO algorithm win * 50 * times/ 50 times 
 Average Converge time for DynamicPSO is  337.82 
 Average Converge time for PSO is  472.46 
 which is  134.64 steps quicker!


In [479]:
#Script
def test_dyn(f,dim,population,itr):
    
    d = [('Converge','bool'),('Time','float'),('Dim','int'),('w_e','float'),('a1','float'),('a2','float'),('population_size','int'),('time_step','int'),('search_range','float')]
    values =np.zeros(itr,dtype=d)
    for i in range(itr):
        a = np.random.choice(np.arange(4.02,4.09,0.1)) # [4.0, 4.4]
#         dim = np.random.choice(np.arange(5,51,5)) # [5, 50]
        dim = dim
#         w_s = round(np.random.choice(np.arange(0.80,0.86,0.01)), 2) # [0.80, 0.85]
        w_e = round(np.random.choice(np.arange(0.80,0.86,0.01)), 2)# [0.80, 0.85]
        a1 = round(a/2,2)# [0.9, 1.3]
        a2 = round(a/2,2)
#         population_size = np.random.choice(np.arange(20,50,10)) # [20, 40]
        population_size = population
        search_range = 5.12
        time_steps = 5001
        function = str(f.__name__)
        result = PSODynamic(w_e, a1, a2, dim, population_size, time_steps, search_range, func=f).run()
        print('Iteration = ',i,' dim=',dim,', w=',w_e,', a1 =',a1,', a2=',a2,', population_size=', population_size)
        v = np.array([(result[2],result[1],dim,w_e,a1,a2,population_size,time_steps,search_range)],dtype=d)
#         print(function)
#         print(v)
#         values = np.append(values,v)
        values[i] = v
#     print(values)
    values = np.sort(np.array(values),order = 'Time')
    print(function," function sorted result:\n",values)
    return values

In [475]:
output_sphere_dyn = test_dyn(sphere,50,40,itr=50)

Iteration =  0  dim= 50 , w= 0.8 , a1 = 2.01 , a2= 2.01 , population_size= 40
Iteration =  1  dim= 50 , w= 0.85 , a1 = 2.01 , a2= 2.01 , population_size= 40
Iteration =  2  dim= 50 , w= 0.82 , a1 = 2.01 , a2= 2.01 , population_size= 40
Iteration =  3  dim= 50 , w= 0.81 , a1 = 2.01 , a2= 2.01 , population_size= 40
Iteration =  4  dim= 50 , w= 0.83 , a1 = 2.01 , a2= 2.01 , population_size= 40
Iteration =  5  dim= 50 , w= 0.85 , a1 = 2.01 , a2= 2.01 , population_size= 40
Iteration =  6  dim= 50 , w= 0.85 , a1 = 2.01 , a2= 2.01 , population_size= 40
Iteration =  7  dim= 50 , w= 0.84 , a1 = 2.01 , a2= 2.01 , population_size= 40
Iteration =  8  dim= 50 , w= 0.84 , a1 = 2.01 , a2= 2.01 , population_size= 40
Iteration =  9  dim= 50 , w= 0.83 , a1 = 2.01 , a2= 2.01 , population_size= 40
Iteration =  10  dim= 50 , w= 0.84 , a1 = 2.01 , a2= 2.01 , population_size= 40
Iteration =  11  dim= 50 , w= 0.83 , a1 = 2.01 , a2= 2.01 , population_size= 40
Iteration =  12  dim= 50 , w= 0.85 , a1 = 2.01 , a2

In [480]:
output_ras_dyn = test_dyn(rastrigin,4,120,itr=50)

Iteration =  0  dim= 4 , w= 0.84 , a1 = 2.01 , a2= 2.01 , population_size= 120
Iteration =  1  dim= 4 , w= 0.85 , a1 = 2.01 , a2= 2.01 , population_size= 120
Iteration =  2  dim= 4 , w= 0.8 , a1 = 2.01 , a2= 2.01 , population_size= 120
Iteration =  3  dim= 4 , w= 0.85 , a1 = 2.01 , a2= 2.01 , population_size= 120
Iteration =  4  dim= 4 , w= 0.8 , a1 = 2.01 , a2= 2.01 , population_size= 120
Iteration =  5  dim= 4 , w= 0.83 , a1 = 2.01 , a2= 2.01 , population_size= 120
Iteration =  6  dim= 4 , w= 0.83 , a1 = 2.01 , a2= 2.01 , population_size= 120
Iteration =  7  dim= 4 , w= 0.8 , a1 = 2.01 , a2= 2.01 , population_size= 120
Iteration =  8  dim= 4 , w= 0.8 , a1 = 2.01 , a2= 2.01 , population_size= 120
Iteration =  9  dim= 4 , w= 0.83 , a1 = 2.01 , a2= 2.01 , population_size= 120
Iteration =  10  dim= 4 , w= 0.81 , a1 = 2.01 , a2= 2.01 , population_size= 120
Iteration =  11  dim= 4 , w= 0.82 , a1 = 2.01 , a2= 2.01 , population_size= 120
Iteration =  12  dim= 4 , w= 0.83 , a1 = 2.01 , a2= 2.

In [481]:
table_sphere_dyn = PrettyTable(['Converge', 'Converge Time','Dim','w_end','a1','a2','Population','Time_step','Range'])
[table_sphere_dyn.add_row(output_sphere_dyn[i]) for i in range(output_sphere_dyn.shape[0])]
print(table_sphere_dyn)

+----------+---------------+-----+-------+------+------+------------+-----------+-------+
| Converge | Converge Time | Dim | w_end |  a1  |  a2  | Population | Time_step | Range |
+----------+---------------+-----+-------+------+------+------------+-----------+-------+
|   True   |     1591.0    |  50 |  0.8  | 2.01 | 2.01 |     40     |    5001   |  5.12 |
|   True   |     1759.0    |  50 |  0.8  | 2.01 | 2.01 |     40     |    5001   |  5.12 |
|   True   |     1836.0    |  50 |  0.8  | 2.01 | 2.01 |     40     |    5001   |  5.12 |
|   True   |     1906.0    |  50 |  0.81 | 2.01 | 2.01 |     40     |    5001   |  5.12 |
|   True   |     2017.0    |  50 |  0.82 | 2.01 | 2.01 |     40     |    5001   |  5.12 |
|   True   |     2032.0    |  50 |  0.82 | 2.01 | 2.01 |     40     |    5001   |  5.12 |
|   True   |     2037.0    |  50 |  0.81 | 2.01 | 2.01 |     40     |    5001   |  5.12 |
|   True   |     2075.0    |  50 |  0.81 | 2.01 | 2.01 |     40     |    5001   |  5.12 |
|   True  

In [482]:
table_ras_dyn = PrettyTable(['Converge', 'Converge Time','Dim','w_end','a1','a2','Population','Time_step','Range'])
[table_ras_dyn.add_row(output_ras_dyn[i]) for i in range(output_ras_dyn.shape[0])]
print(table_ras_dyn)

+----------+---------------+-----+-------+------+------+------------+-----------+-------+
| Converge | Converge Time | Dim | w_end |  a1  |  a2  | Population | Time_step | Range |
+----------+---------------+-----+-------+------+------+------------+-----------+-------+
|   True   |     144.0     |  4  |  0.81 | 2.01 | 2.01 |    120     |    5001   |  5.12 |
|   True   |     145.0     |  4  |  0.8  | 2.01 | 2.01 |    120     |    5001   |  5.12 |
|   True   |     151.0     |  4  |  0.8  | 2.01 | 2.01 |    120     |    5001   |  5.12 |
|   True   |     164.0     |  4  |  0.8  | 2.01 | 2.01 |    120     |    5001   |  5.12 |
|   True   |     168.0     |  4  |  0.8  | 2.01 | 2.01 |    120     |    5001   |  5.12 |
|   True   |     173.0     |  4  |  0.84 | 2.01 | 2.01 |    120     |    5001   |  5.12 |
|   True   |     181.0     |  4  |  0.81 | 2.01 | 2.01 |    120     |    5001   |  5.12 |
|   True   |     186.0     |  4  |  0.82 | 2.01 | 2.01 |    120     |    5001   |  5.12 |
|   True  

In [185]:
a = 4.08
PSODynamic(dim=5, w_s=0.86, w_e=0.75, a1=a/2, a2=a/2, population_size=150, time_steps=2001, search_range=5.12, func=rastrigin).run()

init
Time:      0,  Best Fitness:      30.944394,  Best Pos:    0.2720,   2.0424 ...
Time:    100,  Best Fitness:       1.152846,  Best Pos:    0.0047,   0.0047 ...
Time:    200,  Best Fitness:       0.000000,  Best Pos:    0.0000,   0.0000 ...
Converged at iteration 212


# Problem 2: GP

In [638]:
#Generate dataset
def generate_dataset(target_func, n_samples): # generate data points from target_func
    DATASET_SIZE = n_samples
    data_seed = np.random.uniform(low=-5.12, high=5.12, size = (DATASET_SIZE,2))
    return [[x[0],x[1],target_func(x,2)] for x in data_seed]

In [639]:
from random import random, randint, seed, choice
from statistics import mean
from copy import deepcopy

In [652]:
def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y
#def div(x,y): return x/y # Consider what issues might arrise with this function
def sin(x): return math.sin(x)
def cos(x): return math.cos(x)
def pow_2(x): return x ** 2
# def protected_div(x, y):
#     return x/(y+1e-5)

In [653]:
import sys
class GPTree:
    def __init__(self, data = None, left = None, right = None):
        self.data  = data
        self.left  = left # left subtree
        self.right = right # right subtree
        self.height = 0 # height of the whole tree, which should be constrained
        
        
    def node_label(self): # string label
        if (self.data in FUNCTIONS):
            return self.data.__name__
        else: 
            if self.data == math.pi:
                return 'pi'
            else:
                return str(self.data)
    
    def print_tree(self, prefix = ""): # textual printout
#         print("%s%s" % (prefix, self.node_label()))        
        if self.left:  self.left.print_tree (prefix + "   ")
        if self.right and self.data not in FUNCTIONS1: self.right.print_tree(prefix + "   ")

    def compute_tree(self, x, y): 
        try:
            if (self.data in FUNCTIONS2): 
                return self.data(self.left.compute_tree(x, y), self.right.compute_tree(x, y))
            if (self.data in FUNCTIONS1): # just take the value of left subtree if function is unary
                return self.data(self.left.compute_tree(x, y))
            elif self.data == 'x': return x
            elif self.data == 'y': return y
            else: return self.data
        # ignore subtree with exception
        except TypeError:
            return 0
        except OverflowError:
            return float('inf')
        except ValueError:
            return 0
        except AttributeError:
            self.print_tree()
            sys.exit()
            
    def choose_fuction(self):
        data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
#         if data == pow_type1:
#             data = choice([pow2,pow3,pow4])
#         elif data == pow_type2:
#             data = choice([pow12,pow13,pow14])
        return data
    def random_tree(self, grow, max_depth, depth = 0): # create random tree using either grow or full method
        if depth < MIN_DEPTH or (depth < max_depth and not grow): 
            self.data = self.choose_fuction()
        elif depth >= max_depth:   
            self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
        else: # intermediate depth, grow
            if random () > 0.5: 
                self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
            else:
                self.data = self.choose_fuction()
                
        if self.data in FUNCTIONS2:
            self.left = GPTree()          
            self.left.random_tree(grow, max_depth, depth = depth + 1)    
            self.right = GPTree()
            self.right.random_tree(grow, max_depth, depth = depth + 1)
        elif self.data in FUNCTIONS1:
            self.left = GPTree()          
            self.left.random_tree(grow, max_depth, depth = depth + 1)   
        self.update_height()

    def mutation(self):
        if random() < PROB_MUTATION: # mutate at this node
            self.random_tree(grow = True, max_depth = 2)
        elif self.left: self.left.mutation() #mutate left child
        elif self.right: self.right.mutation() # mutate right child
        self.update_height()

    def size(self): # tree size in nodes
        if self.data in TERMINALS: return 1
        l = self.left.size()  if self.left  else 0
        r = self.right.size() if self.right else 0
        return 1 + l + r
    def update_height(self): # udpate the height of the whole tree
        if self.data in TERMINALS:
            self.height = 1
        else:
            tmp = 0
            if self.left:
                tmp = max(tmp, self.left.update_height())
            if self.right:
                tmp = max(tmp, self.right.update_height())
            self.height = 1 + tmp
        return self.height

    def build_subtree(self): # count is list in order to pass "by reference"
        t = GPTree()
        t.data = self.data
        if self.left:  t.left  = self.left.build_subtree()
        if self.right: t.right = self.right.build_subtree()
        return t
                        
    def scan_tree(self, count, second): # note: count is list, so it's passed "by reference"
        count[0] -= 1            
        if count[0] <= 1: 
            if not second: # return subtree rooted here
                return self.build_subtree()
            else: # glue subtree here
                self.data  = second.data
                self.left  = second.left
                self.right = second.right
        else:  
            ret = None              
            if self.left  and count[0] > 1: ret = self.left.scan_tree(count, second)  
            if self.right and count[0] > 1: ret = self.right.scan_tree(count, second)  
            return ret

    def crossover(self, other): # xo 2 trees at random nodes
        if random() < XO_RATE:
            second = other.scan_tree([randint(1, other.size())], None) # 2nd random subtree
            self.scan_tree([randint(1, self.size())], second) # 2nd subtree "glued" inside 1st tree
            self.update_height()

In [654]:
def fitness(individual, dataset): # inverse mean absolute error over dataset normalized to [0,1]
    try:
        return 1 / (1 + mean([abs(individual.compute_tree(ds[0],ds[1]) - ds[2]) for ds in dataset]))
    except OverflowError:
        return 0

In [655]:
def selection(population, fitnesses): # select one individual using tournament selection
    tournament = [randint(0, len(population)-1) for i in range(TOURNAMENT_SIZE)] # select tournament contenders
    tournament_fitnesses = []
    for i in range(TOURNAMENT_SIZE):
        tournament_fitnesses.append(fitnesses[tournament[i]])
    return deepcopy(population[tournament[tournament_fitnesses.index(max(tournament_fitnesses))]]) 

In [656]:
def init_population(): # ramped half-and-half
    pop = []
    for md in range(MIN_DEPTH, MAX_DEPTH + 1):
        for i in range(int(POP_SIZE/((MAX_DEPTH-MIN_DEPTH+1)*2))):
            t = GPTree()
            t.random_tree(grow = True, max_depth = md) # grow
            pop.append(t) 
        for i in range(int(POP_SIZE/((MAX_DEPTH-MIN_DEPTH+1)*2))):
            t = GPTree()
            t.random_tree(grow = False, max_depth = md) # full
            pop.append(t) 
    return pop
def runGP(dataset,n_hall_of_fame=1,max_h=13):
    population = init_population() 
    best_of_run = None
    best_of_run_f = 0
    best_of_run_gen = 0
    fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]

    # go evolution!
    for gen in range(GENERATIONS):        
        nextgen_population=[]
        if gen%50 == 0:
            print("Gen",gen,"Fitness",best_of_run_f)
        for each in population:
            if each.height>=max_h:
                population.remove(each)
        fitnesses = [fitness(population[i], dataset) for i in range(len(population))]
        topK = np.argpartition(fitnesses, -n_hall_of_fame)[-n_hall_of_fame:]
        hall_of_fame = [population[i] for i in range(len(population)) if fitnesses[i] in topK]
        if len(hall_of_fame)>0:
            nextgen_population.extend(choices(hall_of_fame,k=n_hall_of_fames))
        for i in range(POP_SIZE-len(nextgen_population)):
            parent1 = selection(population, fitnesses)
            parent2 = selection(population, fitnesses)
            parent1.crossover(parent2)
            parent1.mutation()
            nextgen_population.append(parent1)
        
        population = nextgen_population
        fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]
        if max(fitnesses) > best_of_run_f:
            dif = max(fitnesses) - best_of_run_f
            best_of_run_f = max(fitnesses)
            best_of_run_gen = gen
            best_of_run = deepcopy(population[fitnesses.index(max(fitnesses))])
            
            if dif > 0.03:
                print("________________________")
                print("gen:", gen, ", best_of_run_f:", round(max(fitnesses),3), ", best_of_run:") 
#             best_of_run.print_tree()
        if abs(best_of_run_f - 1)<1e-5: break   

    print("\n\n*********************************\nEND OF RUN\nbest_of_run attained at gen " + str(best_of_run_gen) +\
            " and has f=" + str(round(best_of_run_f,3)))
    best_of_run.print_tree()
    return best_of_run,best_of_run_f,best_of_run_gen

In [657]:
FUNCTIONS = [add, sub, mul, sin, cos, pow_2]
FUNCTIONS1 = [sin, cos, pow_2] # unary operators
FUNCTIONS2 = [add, sub, mul] # binary operators
TERMINALS = ['x', 'y', 0, 1, 2, 5, 10, math.pi]
POP_SIZE        = 120   # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 3    # maximal initial random tree depth
GENERATIONS     = 500  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
XO_RATE         = 0.9  # crossover rate 
PROB_MUTATION   = 0.2  # per-node mutation probability 
dataset1 = generate_dataset(sphere, 50)
runGP(dataset1)

Gen 0 Fitness 0
________________________
gen: 0 , best_of_run_f: 0.135 , best_of_run:
________________________
gen: 7 , best_of_run_f: 1.0 , best_of_run:


*********************************
END OF RUN
best_of_run attained at gen 7 and has f=1.0


(<__main__.GPTree at 0x7f832c2eb340>, 1.0, 7)

In [ ]:
FUNCTIONS = [add, sub, mul, sin, cos, pow_2]
FUNCTIONS1 = [sin, cos, pow_2]
FUNCTIONS2 = [add, sub, mul]
TERMINALS = ['x', 'y', 0, 1, 2, 5, 10, math.pi]
POP_SIZE        = 80   # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 6    # maximal initial random tree depth
GENERATIONS     = 400  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 6    # size of tournament for tournament selection
XO_RATE         = 0.90  # crossover rate 
PROB_MUTATION   = 0.40  # per-node mutation probability 
dataset2 = generate_dataset(rastrigin, 50)
runGP(dataset2)

In [645]:
def test_gp_sph(f,dim,itr):
    d = [('DataGenerate','int'),('POP_SIZE','int'),('MIN_DEPTH','int'),('MAX_DEPTH','int'),('GENERATIONS','int'),
         ('TOURNAMENT_SIZE','int'),('XO_RATE','float'),('PROB_MUTATION','float'),('Best_fitness','float'),('Best_gen',int)]
    values =np.zeros(itr,dtype=d)
    for i in range(itr):
        print('iteration = ', i)
#         POP_SIZE        = 120   # population size
#         MIN_DEPTH       = 2    # minimal initial random tree depth
#         MAX_DEPTH       = 6    # maximal initial random tree depth
#         GENERATIONS     = 250  # maximal number of generations to run evolution
#         TOURNAMENT_SIZE = 4    # size of tournament for tournament selection
#         XO_RATE         = 0.75  # crossover rate 
#         PROB_MUTATION   = 0.15  # per-node mutation probability 
        POP_SIZE = np.random.choice(np.arange(100,141,10)) # [-40, 40]
        MIN_DEPTH       = 2    # minimal initial random tree depth
        MAX_DEPTH       = 6    # maximal initial random tree depth
        GENERATIONS = np.random.choice(np.arange(100,251,50),1)
        TOURNAMENT_SIZE = np.random.choice(np.arange(3,6,1)) # [1,5]
        XO_RATE = round(np.random.choice(np.arange(0.70,0.81,0.05)),2) # [0.5, 0.95]
        PROB_MUTATION = round(np.random.choice(np.arange(0.15,0.35,0.05)),2) # [0.15, 0.30]
        
        function = str(f.__name__)
        dataset2 = generate_dataset(f, dim)
        _,best_of_run_f,best_of_run_gen = runGP(dataset2)
#         print('No.Data = ',dim,' Pop =',POP_SIZE,', GENERATIONS =',GENERATIONS,
#               ', XO_RATE = ',XO_RATE,', TOURNAMENT=', TOURNAMENT_SIZE,'Best_fitness=',best_of_run_f,'Best_gen=',best_of_run_gen)
        v = np.array([(dim,POP_SIZE,MIN_DEPTH,MAX_DEPTH,GENERATIONS,TOURNAMENT_SIZE,XO_RATE,PROB_MUTATION,round(best_of_run_f,2),best_of_run_gen)],dtype=d)
        values[i] = v
#     print(values)
    values = np.sort(np.array(values),order = 'Best_gen')
    print(function," function sorted result:\n",values)
    return values

In [591]:
output_sph_gp = test_gp_sph(sphere,50,50)

iteration =  0
add
   sin
      add
         mul
            pi
            5
         add
            1
            0
   add
      add
         pow_2
            x
         pow_2
            y
      sin
         1
iteration =  1
add
   pow_2
      x
   pow_2
      y
iteration =  2
add
   pow_2
      x
   sub
      pow_2
         y
      mul
         pow_2
            x
         0
iteration =  3
add
   add
      sin
         0
      pow_2
         x
   pow_2
      y
iteration =  4
add
   pow_2
      x
   pow_2
      y
iteration =  5
add
   sin
      sin
         0
   add
      sin
         sin
            0
      add
         add
            mul
               x
               x
            0
         pow_2
            sub
               0
               y
iteration =  6
add
   pow_2
      y
   pow_2
      x
iteration =  7
add
   pow_2
      sub
         0
         x
   pow_2
      y
iteration =  8
add
   pow_2
      x
   pow_2
      y
iteration =  9
add
   pow_2
      x
   pow_2
     

In [596]:
table_sph_gp = PrettyTable(['Data','POP_SIZE','MIN_D','MAX_D','GENERATIONS','TOUR_SIZE',
                             'XO','PROB','B_fitness','B_gen'])
[table_sph_gp.add_row(output_sph_gp[i]) for i in range(output_sph_gp.shape[0])]
print(table_sph_gp)

+------+----------+-------+-------+-------------+-----------+------+------+-----------+-------+
| Data | POP_SIZE | MIN_D | MAX_D | GENERATIONS | TOUR_SIZE |  XO  | PROB | B_fitness | B_gen |
+------+----------+-------+-------+-------------+-----------+------+------+-----------+-------+
|  50  |   100    |   2   |   6   |     250     |     5     | 0.7  | 0.2  |    1.0    |   0   |
|  50  |   100    |   2   |   6   |     200     |     5     | 0.75 | 0.3  |    1.0    |   1   |
|  50  |   130    |   2   |   6   |     200     |     4     | 0.7  | 0.15 |    1.0    |   1   |
|  50  |   100    |   2   |   6   |     200     |     5     | 0.75 | 0.3  |    1.0    |   2   |
|  50  |   110    |   2   |   6   |     100     |     5     | 0.7  | 0.2  |    1.0    |   2   |
|  50  |   110    |   2   |   6   |     150     |     5     | 0.8  | 0.25 |    1.0    |   2   |
|  50  |   120    |   2   |   6   |     100     |     5     | 0.7  | 0.2  |    1.0    |   2   |
|  50  |   120    |   2   |   6   |     

In [558]:
table_sph_gp = PrettyTable(['Data','POP_SIZE','MIN_D','MAX_D','GENERATIONS','TOUR_SIZE',
                             'XO','PROB','B_fitness','B_gen'])
[table_sph_gp.add_row(output_sph_gp[i]) for i in range(output_sph_gp.shape[0])]
print(table_sph_gp)

+------+----------+-------+-------+-------------+-----------+------+------+-----------+-------+
| Data | POP_SIZE | MIN_D | MAX_D | GENERATIONS | TOUR_SIZE |  XO  | PROB | B_fitness | B_gen |
+------+----------+-------+-------+-------------+-----------+------+------+-----------+-------+
|  50  |   130    |   2   |   6   |     100     |     4     | 0.75 | 0.25 |    1.0    |   0   |
|  50  |   130    |   2   |   6   |     150     |     3     | 0.8  | 0.2  |    1.0    |   1   |
|  50  |   130    |   2   |   6   |     200     |     5     | 0.75 | 0.25 |    1.0    |   1   |
|  50  |   130    |   2   |   6   |     100     |     3     | 0.75 | 0.25 |    1.0    |   2   |
|  50  |   130    |   2   |   6   |     100     |     3     | 0.8  | 0.2  |    1.0    |   2   |
|  50  |   130    |   2   |   6   |     100     |     4     | 0.7  | 0.3  |    1.0    |   2   |
|  50  |   140    |   2   |   6   |     150     |     5     | 0.75 | 0.25 |    1.0    |   2   |
|  50  |   100    |   2   |   6   |     

In [619]:
#Fig2
from sklearn.linear_model import LinearRegression
# print([list(output_sph_gp[i])[:-1] for i in range(output_sph_gp.shape[0])])
X = [list(output_sph_gp[i])[:-1] for i in range(output_sph_gp.shape[0])]
y = [list(output_sph_gp[i])[-1] for i in range(output_sph_gp.shape[0])]
reg = LinearRegression().fit(X, y)
coef = reg.coef_
# print([round(coef[j],3) for j in range(len(coef))])
table_sph_gp = PrettyTable(['coef_Data','coe_POPSIZE','cMIN_D','cMAX_D','coef_GENS','coef_TOURSIZE',
                             'coef_XO','coef_PROB','coef_B_f'])
table_sph_gp.add_row([round(coef[j],3) for j in range(len(coef))])
print(table_sph_gp)

[[50, 100, 2, 6, 250, 5, 0.7, 0.2], [50, 100, 2, 6, 200, 5, 0.75, 0.3], [50, 130, 2, 6, 200, 4, 0.7, 0.15], [50, 100, 2, 6, 200, 5, 0.75, 0.3], [50, 110, 2, 6, 100, 5, 0.7, 0.2], [50, 110, 2, 6, 150, 5, 0.8, 0.25], [50, 120, 2, 6, 100, 5, 0.7, 0.2], [50, 120, 2, 6, 250, 3, 0.8, 0.25], [50, 100, 2, 6, 100, 4, 0.8, 0.2], [50, 100, 2, 6, 150, 4, 0.75, 0.2], [50, 100, 2, 6, 150, 5, 0.75, 0.15], [50, 140, 2, 6, 150, 3, 0.7, 0.15], [50, 100, 2, 6, 250, 3, 0.7, 0.25], [50, 100, 2, 6, 250, 5, 0.7, 0.25], [50, 110, 2, 6, 150, 4, 0.75, 0.2], [50, 120, 2, 6, 100, 5, 0.7, 0.15], [50, 120, 2, 6, 150, 5, 0.8, 0.2], [50, 140, 2, 6, 100, 4, 0.75, 0.25], [50, 140, 2, 6, 100, 5, 0.8, 0.2], [50, 100, 2, 6, 200, 5, 0.8, 0.15], [50, 110, 2, 6, 100, 5, 0.75, 0.3], [50, 120, 2, 6, 100, 5, 0.7, 0.25], [50, 120, 2, 6, 150, 3, 0.7, 0.2], [50, 140, 2, 6, 150, 3, 0.7, 0.15], [50, 120, 2, 6, 150, 4, 0.7, 0.15], [50, 120, 2, 6, 150, 5, 0.8, 0.3], [50, 100, 2, 6, 100, 3, 0.7, 0.15], [50, 100, 2, 6, 100, 4, 0.75, 0.2

In [593]:
def test_gp_ras(f,dim,itr):
    d = [('DataGenerate','int'),('POP_SIZE','int'),('MIN_DEPTH','int'),('MAX_DEPTH','int'),('GENERATIONS','int'),
         ('TOURNAMENT_SIZE','int'),('XO_RATE','float'),('PROB_MUTATION','float'),('Best_fitness','float'),('Best_gen',int)]
    values =np.zeros(itr,dtype=d)
    for i in range(itr):
        print('iteration',i)
# POP_SIZE        = 60   # population size
# MIN_DEPTH       = 2    # minimal initial random tree depth
# MAX_DEPTH       = 5    # maximal initial random tree depth
# GENERATIONS     = 250  # maximal number of generations to run evolution
# TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
# XO_RATE         = 0.70  # crossover rate 
# PROB_MUTATION   = 0.30  # per-node mutation probability 
        POP_SIZE = np.random.choice(np.arange(50,81,10)) # [50, 80]
        MIN_DEPTH       = 2    # minimal initial random tree depth
        MAX_DEPTH       = 5    # maximal initial random tree depth
        GENERATIONS = np.random.choice(np.arange(200,301,20),1)
        TOURNAMENT_SIZE = np.random.choice(np.arange(4,8,1)) # [4,7]
        XO_RATE = round(np.random.choice(np.arange(0.70,0.96,0.05)),2) # [0.7, 0.95]
        PROB_MUTATION = round(np.random.choice(np.arange(0.15,0.35,0.05)),2) # [0.15, 0.30]
        
        function = str(f.__name__)
        dataset2 = generate_dataset(f, dim)
        _,best_of_run_f,best_of_run_gen = runGP(dataset2)
#         print('No.Data = ',dim,' Pop =',POP_SIZE,', GENERATIONS =',GENERATIONS,
#               ', XO_RATE = ',XO_RATE,', TOURNAMENT=', TOURNAMENT_SIZE,'Best_fitness=',best_of_run_f,'Best_gen=',best_of_run_gen)
        v = np.array([(dim,POP_SIZE,MIN_DEPTH,MAX_DEPTH,GENERATIONS,TOURNAMENT_SIZE,XO_RATE,PROB_MUTATION,round(best_of_run_f,2),best_of_run_gen)],dtype=d)
        values[i] = v
#     print(values)
    values = np.sort(np.array(values),order = 'Best_gen')
    print(function," function sorted result:\n",values)
    return values

In [560]:
output_ras_gp = test_gp_ras(rastrigin,50,2)

iteration 0
add
   cos
      mul
         sin
            2
         add
            2
            2
   add
      sin
         add
            mul
               add
                  sin
                     add
                        sin
                           pow_2
                              pow_2
                                 x
                        y
                  add
                     sin
                        add
                           pow_2
                              x
                           pow_2
                              x
                     y
               sin
                  5
            add
               mul
                  pow_2
                     mul
                        pow_2
                           y
                        cos
                           2
                  cos
                     2
               y
      add
         add
            mul
               5
               sin
                  add
   

In [561]:
table_ras_gp = PrettyTable(['Data','POP_SIZE','MIN_D','MAX_D','GENERATIONS','TOUR_SIZE',
                             'XO','PROB','B_fitness','B_gen'])
[table_ras_gp.add_row(output_ras_gp[i]) for i in range(output_ras_gp.shape[0])]
print(table_ras_gp)

+------+----------+-------+-------+-------------+-----------+-----+------+-----------+-------+
| Data | POP_SIZE | MIN_D | MAX_D | GENERATIONS | TOUR_SIZE |  XO | PROB | B_fitness | B_gen |
+------+----------+-------+-------+-------------+-----------+-----+------+-----------+-------+
|  50  |    50    |   2   |   5   |     220     |     5     | 0.6 | 0.4  |    0.16   |  450  |
|  50  |    80    |   2   |   5   |     240     |     5     | 0.7 | 0.3  |    0.16   |  475  |
+------+----------+-------+-------+-------------+-----------+-----+------+-----------+-------+


In [597]:
output_ras_gp_2 = test_gp_ras(rastrigin,50,20)

iteration 0
add
   add
      sin
         mul
            mul
               pow_2
                  5
               add
                  pi
                  y
            mul
               y
               1
      sub
         x
         cos
            y
   add
      add
         sin
            add
               cos
                  mul
                     mul
                        5
                        1
                     mul
                        mul
                           5
                           mul
                              2
                              mul
                                 5
                                 1
                        y
               cos
                  mul
                     mul
                        mul
                           1
                           mul
                              2
                              mul
                                 5
                                 1
          

add
   add
      mul
         sin
            10
         sin
            10
      add
         sin
            add
               add
                  y
                  add
                     add
                        add
                           add
                              sin
                                 mul
                                    sub
                                       sub
                                          2
                                          5
                                       mul
                                          0
                                          1
                                    2
                              add
                                 sin
                                    add
                                       add
                                          y
                                          add
                                             add
                                   

add
   cos
      cos
         add
            pow_2
               sub
                  mul
                     0
                     1
                  add
                     10
                     2
            mul
               pi
               x
   add
      add
         cos
            add
               add
                  cos
                     pow_2
                        cos
                           pow_2
                              y
                  10
               add
                  sub
                     sub
                        sub
                           sub
                              pow_2
                                 y
                              sin
                                 y
                           sin
                              sin
                                 y
                        sin
                           sin
                              y
                     sin
                        add
    

add
   sub
      sin
         mul
            10
            y
      cos
         add
            pow_2
               pow_2
                  sin
                     sin
                        mul
                           y
                           pi
            add
               cos
                  cos
                     mul
                        x
                        1
               pow_2
                  sin
                     sin
                        mul
                           y
                           pi
   add
      y
      add
         add
            pow_2
               add
                  cos
                     sub
                        sin
                           y
                        cos
                           y
                  pow_2
                     sin
                        x
            add
               mul
                  sin
                     sin
                        mul
                           y
  

add
   sub
      cos
         x
      sin
         pi
   add
      add
         sin
            add
               mul
                  add
                     1
                     2
                  sub
                     y
                     pow_2
                        x
               sub
                  add
                     add
                        pow_2
                           x
                        pow_2
                           y
                     sin
                        mul
                           y
                           pow_2
                              y
                  cos
                     add
                        x
                        y
         sub
            add
               add
                  sin
                     add
                        mul
                           add
                              1
                              2
                           sub
                              y
    

add
   sub
      cos
         add
            add
               mul
                  pi
                  5
               mul
                  y
                  5
            y
      add
         cos
            x
         y
   add
      add
         add
            sub
               cos
                  pow_2
                     x
               x
            pow_2
               x
         add
            add
               add
                  add
                     mul
                        mul
                           cos
                              cos
                                 y
                           pi
                        10
                     sin
                        add
                           add
                              sin
                                 x
                              pow_2
                                 x
                           add
                              10
                              x
     

add
   cos
      mul
         add
            x
            2
         pow_2
            5
   add
      sub
         sub
            sin
               sin
                  sub
                     sin
                        mul
                           add
                              sub
                                 y
                                 5
                              1
                           sin
                              x
                     cos
                        x
            cos
               add
                  sub
                     y
                     sin
                        mul
                           pow_2
                              cos
                                 y
                           sin
                              x
                  pow_2
                     y
         mul
            x
            sin
               x
      add
         add
            sub
               sin
                  sub
   

In [600]:
table_ras_gp_2 = PrettyTable(['Data','POP_SIZE','MIN_D','MAX_D','GENERATIONS','TOUR_SIZE',
                             'XO','PROB','B_fitness','B_gen'])
[table_ras_gp_2.add_row(output_ras_gp_2[i]) for i in range(output_ras_gp_2.shape[0])]
print(table_ras_gp_2)

+------+----------+-------+-------+-------------+-----------+------+------+-----------+-------+
| Data | POP_SIZE | MIN_D | MAX_D | GENERATIONS | TOUR_SIZE |  XO  | PROB | B_fitness | B_gen |
+------+----------+-------+-------+-------------+-----------+------+------+-----------+-------+
|  50  |    50    |   2   |   5   |     200     |     7     | 0.9  | 0.25 |    0.14   |  266  |
|  50  |    80    |   2   |   5   |     300     |     7     | 0.7  | 0.15 |    0.12   |  308  |
|  50  |    70    |   2   |   5   |     200     |     7     | 0.95 | 0.15 |    0.13   |  317  |
|  50  |    80    |   2   |   5   |     300     |     7     | 0.7  | 0.15 |    0.12   |  323  |
|  50  |    70    |   2   |   5   |     200     |     5     | 0.75 | 0.3  |    0.12   |  325  |
|  50  |    80    |   2   |   5   |     300     |     5     | 0.85 | 0.25 |    0.13   |  396  |
|  50  |    70    |   2   |   5   |     200     |     7     | 0.7  | 0.25 |    0.2    |  403  |
|  50  |    80    |   2   |   5   |     

In [620]:
#Fig3
from sklearn.linear_model import LinearRegression
# print([list(output_sph_gp[i])[:-1] for i in range(output_sph_gp.shape[0])])
X1 = [list(output_ras_gp_2[i])[:-2] for i in range(output_ras_gp_2.shape[0])]
y1 = [list(output_ras_gp_2[i])[-2] for i in range(output_ras_gp_2.shape[0])]
reg = LinearRegression().fit(X1, y1)
coef1 = reg.coef_
print([round(coef[j],3) for j in range(len(coef1))])
table_ras_gp = PrettyTable(['coef_Data','coe_POPSIZE','cMIN_D','cMAX_D','coef_GENS','coef_TOURSIZE',
                             'coef_XO','coef_PROB'])
table_ras_gp.add_row([round(coef1[j],3) for j in range(len(coef1))])
print(table_ras_gp)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
+-----------+-------------+--------+--------+-----------+---------------+---------+-----------+
| coef_Data | coe_POPSIZE | cMIN_D | cMAX_D | coef_GENS | coef_TOURSIZE | coef_XO | coef_PROB |
+-----------+-------------+--------+--------+-----------+---------------+---------+-----------+
|    0.0    |    0.001    |  0.0   |  -0.0  |    0.0    |     0.007     |  0.016  |   0.185   |
+-----------+-------------+--------+--------+-----------+---------------+---------+-----------+


# Problem 3: PSO + GP

In [710]:
from copy import deepcopy
from random import choices
def homo_crossover2(tree1, tree2, a1=0.5, a2=0.5):
    if tree1 is None and tree2 is None:
        return None
    elif tree1 in FUNCTIONS and tree2 in FUNCTIONS:
        if tree1 in FUNCTIONS1 and tree2 in FUNCTIONS2:
            return deepcopy(tree1)
        elif tree2 in FUNCTIONS1 and tree1 in FUNCTIONS2:
            return deepcopy(tree2)
        else:
            return GPTree(choices([tree1.data,tree2.data],weights=[a1,a2])[0],
                          homo_crossover2(tree1.left,tree2.left,a1,a2),
                          homo_crossover2(tree1.right,tree2.right,a1,a2))
    else:
        return deepcopy(choices([tree1,tree2],weights=[a1,a2])[0])

In [659]:
class GPSOParticle(Particle):
    def __init__(self, dataset, max_depth, func):
        t = GPTree()
        t.random_tree(grow = choice([True,False]), max_depth = max_depth) # grow
        self.position = t
        self.best_particle_pos = self.position # local optima
        
        self.fit_func = func
        self.dataset = dataset
        self.fitness = self.fit_func(self.position,self.dataset)
        self.best_particle_fitness = self.fitness   # we couldd start with very large number here, 
                                                    #but the actual value is better in case we are lucky 
                
    def setPos(self, pos):
        self.position = deepcopy(pos)
        self.fitness = self.fit_func(self.position,self.dataset)
        if self.fitness > self.best_particle_fitness:     # to update the personal best both 
                                                        # position (for velocity update) and
                                                        # fitness (the new standard) are needed
                                                        # global best is update on swarm leven
            self.best_particle_fitness = self.fitness
            self.best_particle_pos = pos

In [692]:
class GPSO(PSO):
    def __init__(self, a1, a2, a3, population_size, time_steps, dataset, func):

        self.a1 = a1 # Inertia
        self.a2 = a2 # Attraction to personal best
        self.a3 = a3 # Attraction to global best
        self.fitness_func = func
        self.dataset = dataset
        self.swarm = [GPSOParticle(dataset, choice(range(MIN_DEPTH,MAX_DEPTH)), func) for i in range(population_size)]
        self.time_steps = time_steps
        print('init')

        # Initialising global best, you can wait until the end of the first time step
        # but creating a random initial best and fitness which is very high will mean you
        # do not have to write an if statement for the one off case
        self.best_swarm_pos = self.swarm[0].position
        self.best_swarm_fitness = -1

    def particle_newpos(self, p): # return the new position of a particle
        particle = self.swarm[p]
        t1 = particle.position
        t2 = deepcopy(particle.best_particle_pos)
        t3 = deepcopy(self.best_swarm_pos)
        t = homo_crossover2(t1,t2,self.a1/(self.a1+self.a2),self.a2/(self.a1+self.a2))
        t = homo_crossover2(t,t3,(self.a1+self.a2)/(self.a1+self.a2+self.a3),self.a3/(self.a1+self.a2+self.a3))
        return t
    def run(self):
        convergence_count = 0
        for t in range(self.time_steps):
            for p in range(len(self.swarm)):
                new_position = self.particle_newpos(p) # t is the time step now
                self.swarm[p].setPos(new_position)
                new_fitness = self.fitness_func(new_position,self.dataset)
                if new_fitness < 1e-14:
                    print('Diverge at Time:', t,' Best Fit:',self.best_swarm_fitness)
#                     raise SystemExit('Most likely divergent: Decrease parameter values')
                    return self.best_swarm_fitness,self.time_steps,False

                if new_fitness > self.best_swarm_fitness:   # to update the global best both 
                                                            # position (for velocity update) and
                                                            # fitness (the new group norm) are needed
                    self.best_swarm_fitness = new_fitness
                    self.best_swarm_pos = new_position
                if random()<PROB_MUTATION:
                    self.swarm[p].position.mutation()
                
            if t % 100 == 0: #we print only two components even it search space is high-dimensional
                print("Time: %6d,  Best Fitness: %14.6f" % (t,self.best_swarm_fitness))
            if self.best_swarm_fitness>1-1e-5:
                convergence_count+=1
                if convergence_count>=10:
                    print("Converged at iteration %d"%t)
                    return self.best_swarm_fitness,convergence_count,True
        self.best_swarm_pos.print_tree()

In [735]:
# Self generated model function depth = 3
def mydatafunction(x):
    return 2*x**2 - 10*x

In [736]:
dataset_a = generate_dataset(mydatafunction, 50)
with open('sample_data_file.txt','r') as f:
    for line in f:
        (x,y,f_xy) = map(float,line.split(' '))
        dataset.append([x,y,f_xy])
f.close()
FUNCTIONS = [add, sub, mul, sin, cos, pow_2]
FUNCTIONS1 = [sin, cos, pow_2] # unary operators
FUNCTIONS2 = [add, sub, mul] # binary operators
TERMINALS = ['x', 'y', 0, 1, 2, 5, 10, math.pi]

MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 3    # maximal initial random tree depth
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
PROB_MUTATION   = 0.4  # per-node mutation probability 
a=0.6
GPSO(a1=1-a, a2=a/2, a3=a/2, population_size=40, time_steps=5001, dataset=dataset_a,func=fitness).run()

TypeError: mydatafunction() takes 1 positional argument but 2 were given

In [ ]:
# Single run 
dataset_b = generate_dataset(rastrigin,30)
FUNCTIONS = [add, sub, mul, sin, cos, pow_2]
FUNCTIONS1 = [sin, cos, pow_2] # unary operators
FUNCTIONS2 = [add, sub, mul] # binary operators
TERMINALS = ['x', 'y', 0, 1, 2, 5, 10, math.pi]
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 3    # maximal initial random tree depth
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
PROB_MUTATION   = 0.4  # per-node mutation probability 
a=0.6
GPSO(a1=1-a, a2=a/2, a3=a/2, population_size=40, time_steps=1501, dataset=dataset_b,func=fitness).run()

In [721]:
# Test the behaviour of 50 random initialized sets of parameter on the GPSO()

def test_gpso(f,dataset_gen,pop,itr):
    d = [('Convergence',bool),('Converge Time','int'),('a','float'),('POP_SIZE','int')
         ,('PROB_MUTATION','float'),('Best_fitness','float')]
    values =np.zeros(itr,dtype=d)
    for i in range(itr):
        print('iteration',i)
        POP_SIZE = np.random.choice(np.arange(50,81,10)) # [50, 80]
        a = round(np.random.choice(np.arange(0.1,0.9,0.1)),1)
        population_size = int(pop + np.random.choice(np.arange(-40,41,10),1) )
        MIN_DEPTH       = 2    # minimal initial random tree depth
        MAX_DEPTH       = 3    # maximal initial random tree depth
        TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
        
        PROB_MUTATION = round(np.random.choice(np.arange(0.10,0.51,0.1)),2) # [0.10, 0.50]
          
        best_swarm_f,t,Converge = GPSO(a1=1-a, a2=a/2, a3=a/2, population_size=population_size, time_steps=5001, dataset=dataset_gen,func=fitness).run()#         print('No.Data = ',dim,' Pop =',POP_SIZE,', GENERATIONS =',GENERATIONS,
#               ', XO_RATE = ',XO_RATE,', TOURNAMENT=', TOURNAMENT_SIZE,'Best_fitness=',best_of_run_f,'Best_gen=',best_of_run_gen)
        print(best_swarm_f)
        v = np.array([(Converge,t,a,population_size,PROB_MUTATION,best_swarm_f)],dtype=d)
        values[i] = v
#     print(values)
    values = np.sort(np.array(values),order = 'Best_fitness')
    print(str(f.__name__),"gpos function sorted result:\n",values)
    return values

In [723]:
dataset_gen = generate_dataset(sphere, 50)
output_sph_gpso = test_gpso(sphere,dataset_gen,100,50)

iteration 0
init
Time:      0,  Best Fitness:       0.105540
Time:    100,  Best Fitness:       0.343726
Time:    200,  Best Fitness:       0.993071
Converged at iteration 246
1.0
iteration 1
init
Time:      0,  Best Fitness:       0.105540
Converged at iteration 36
1.0
iteration 2
init
Time:      0,  Best Fitness:       0.105540
Time:    100,  Best Fitness:       0.295324
Time:    200,  Best Fitness:       0.295324
Time:    300,  Best Fitness:       0.295324
Diverge at Time: 313  Best Fit: 0.2953244503802418
0.2953244503802418
iteration 3
init
Time:      0,  Best Fitness:       0.096954
Converged at iteration 17
1.0
iteration 4
init
Time:      0,  Best Fitness:       0.118959
Converged at iteration 95
1.0
iteration 5
init
Time:      0,  Best Fitness:       0.108247
Time:    100,  Best Fitness:       0.647660
Converged at iteration 129
1.0
iteration 6
init
Time:      0,  Best Fitness:       0.111850
Converged at iteration 37
1.0
iteration 7
init
Time:      0,  Best Fitness:       0.094

Time:    800,  Best Fitness:       0.235882
Converged at iteration 882
1.0
iteration 38
init
Time:      0,  Best Fitness:       0.118128
Time:    100,  Best Fitness:       0.333333
Time:    200,  Best Fitness:       0.333333
Converged at iteration 272
1.0
iteration 39
init
Time:      0,  Best Fitness:       0.108767
Converged at iteration 46
0.9999999999999987
iteration 40
init
Time:      0,  Best Fitness:       0.103788
Converged at iteration 55
1.0
iteration 41
init
Time:      0,  Best Fitness:       0.118321
Time:    100,  Best Fitness:       0.988126
Time:    200,  Best Fitness:       1.000000
Converged at iteration 207
1.0
iteration 42
init
Time:      0,  Best Fitness:       0.092185
Time:    100,  Best Fitness:       0.602210
Converged at iteration 115
1.0
iteration 43
init
Time:      0,  Best Fitness:       0.105540
Diverge at Time: 89  Best Fit: 0.5104842555345649
0.5104842555345649
iteration 44
init
Time:      0,  Best Fitness:       0.092036
Diverge at Time: 30  Best Fit: 0.1

In [724]:
table_sph_gpso = PrettyTable(['Convergence','Converge Time','a','POP_SIZE',
                              'PROB_MUTATION','Best_fitness'])
[table_sph_gpso.add_row(output_sph_gpso[i]) for i in range(output_sph_gpso.shape[0])]
print(table_sph_gpso)

+-------------+---------------+-----+----------+---------------+---------------------+
| Convergence | Converge Time |  a  | POP_SIZE | PROB_MUTATION |     Best_fitness    |
+-------------+---------------+-----+----------+---------------+---------------------+
|    False    |      5001     | 0.1 |    90    |      0.2      | 0.11408227870233723 |
|    False    |      5001     | 0.2 |   110    |      0.3      | 0.11880946143523102 |
|    False    |      5001     | 0.5 |   120    |      0.4      | 0.12616348968914481 |
|    False    |      5001     | 0.1 |   130    |      0.3      |  0.2953244503802418 |
|    False    |      5001     | 0.7 |   140    |      0.3      |  0.5104842555345649 |
|     True    |       10      | 0.6 |   130    |      0.3      |  0.9999999999999987 |
|     True    |       10      | 0.7 |   100    |      0.4      |  0.9999999999999996 |
|     True    |       10      | 0.1 |    60    |      0.5      |         1.0         |
|     True    |       10      | 0.1 |    70

In [ ]:
#Fig3
from sklearn.linear_model import LinearRegression
# print([list(output_sph_gp[i])[:-1] for i in range(output_sph_gp.shape[0])])
X1 = [list(output_ras_gp_2[i])[:-2] for i in range(output_ras_gp_2.shape[0])]
y1 = [list(output_ras_gp_2[i])[-2] for i in range(output_ras_gp_2.shape[0])]
reg = LinearRegression().fit(X1, y1)
coef1 = reg.coef_
print([round(coef[j],3) for j in range(len(coef1))])
table_ras_gp = PrettyTable(['coef_Data','coe_POPSIZE','cMIN_D','cMAX_D','coef_GENS','coef_TOURSIZE',
                             'coef_XO','coef_PROB'])
table_ras_gp.add_row([round(coef1[j],3) for j in range(len(coef1))])
print(table_ras_gp)

In [731]:
from datetime import datetime 
def compareGpso(f,dim,population,itr):
    result = 0
    t1 = []
    t2 = []
    t3 = []
    t4 = []
    op1 = []
    op2 = []
    MIN_DEPTH       = 2    # minimal initial random tree depth
    MAX_DEPTH       = 3    # maximal initial random tree depth
    TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
    PROB_MUTATION = 0.2
    a_gpso = 0.6
    a_pso = 4.08
    for i in range(itr):
        dataset_gen = generate_dataset(f, dim)

        gpso_st = datetime.now() 
        o1= GPSO(a1=1-a_gpso, a2=a_gpso/2, a3=a_gpso/2, population_size=population, time_steps=5001, dataset=dataset_gen,func=fitness).run()
        gpso_et = datetime.now() 
        gpso_runtime = (gpso_et - gpso_st).seconds
        
        o2= PSODynamic(dim=50, w_e =0.82,a1=a_pso/2, a2=a_pso/2, population_size=population, time_steps=5001, search_range=5.12, func=sphere).run()
        
        gp_st=datetime.now() 
        o3= runGP(dataset_gen)
        gp_et=datetime.now() 
        gp_runtime = (gp_et - gp_st).seconds
        
        if (o1[1]<o2[1]): 
            result += 1
        t1.append(o1[1])
        t2.append(o2[1])
        t3.append(gpso_runtime)
        t4.append(gp_runtime)
        op1.append(o1[2])
        op2.append(o2[2])
    a1 = round(np.array(t1).mean(),2)
    a2 = round(np.array(t2).mean(),2)
    a3 = round(np.array(t3).mean(),2)
    a4 = round(np.array(t4).mean(),2)
    step_Dif = round(a2 - a1,2)
    time_Dif = round(a4 - a3,2)
    print('GPSO algorithm win *',result,'* times/',itr)
    print('Average Converge step for GPSO is ',a1,'\n'
          ,'Average Converge step for PSO is ',a2,'\n','GPSO is ', step_Dif , 'steps quicker!')
    print('Within the tests, GPSO/PSO failed to converge for',np.array(op1).mean(),'/',np.array(op2).mean(),'times')
    print('Average runtime for GPSO is ',a3,'\n'
          ,'Average runtime for GP is ',a4,'\n','GPSO is ', time_Dif , 'seconds quicker!')
compareGpso(sphere,50,70,50)

init
Time:      0,  Best Fitness:       0.101567
Time:    100,  Best Fitness:       0.127911
Converged at iteration 155
Gen 0 Fitness 0
________________________
gen: 0 , best_of_run_f: 0.116 , best_of_run:
________________________
gen: 5 , best_of_run_f: 1.0 , best_of_run:


*********************************
END OF RUN
best_of_run attained at gen 5 and has f=1.0
init
Time:      0,  Best Fitness:       0.097683
Time:    100,  Best Fitness:       0.153961
Time:    200,  Best Fitness:       0.156802
Time:    300,  Best Fitness:       0.157360
Time:    400,  Best Fitness:       0.160621
Time:    500,  Best Fitness:       0.160621
Time:    600,  Best Fitness:       0.160621
Time:    700,  Best Fitness:       0.161593
Converged at iteration 712
Gen 0 Fitness 0
________________________
gen: 0 , best_of_run_f: 0.123 , best_of_run:
________________________
gen: 4 , best_of_run_f: 0.167 , best_of_run:
________________________
gen: 7 , best_of_run_f: 1.0 , best_of_run:


*************************

________________________
gen: 16 , best_of_run_f: 1.0 , best_of_run:


*********************************
END OF RUN
best_of_run attained at gen 16 and has f=1.0
init
Time:      0,  Best Fitness:       0.100219
Time:    100,  Best Fitness:       0.146190
Time:    200,  Best Fitness:       0.147379
Time:    300,  Best Fitness:       0.151466
Time:    400,  Best Fitness:       0.151466
Time:    500,  Best Fitness:       0.157138
Time:    600,  Best Fitness:       0.187645
Time:    700,  Best Fitness:       0.187645
Time:    800,  Best Fitness:       0.187645
Time:    900,  Best Fitness:       0.289977
Time:   1000,  Best Fitness:       0.289977
Time:   1100,  Best Fitness:       0.289977
Time:   1200,  Best Fitness:       0.289977
Time:   1300,  Best Fitness:       0.289977
Converged at iteration 1310
Gen 0 Fitness 0
________________________
gen: 0 , best_of_run_f: 0.098 , best_of_run:
________________________
gen: 1 , best_of_run_f: 0.133 , best_of_run:
________________________
gen: 11 ,

Gen 0 Fitness 0
________________________
gen: 0 , best_of_run_f: 0.118 , best_of_run:
Gen 50 Fitness 0.1472755312327635
________________________
gen: 51 , best_of_run_f: 0.197 , best_of_run:
________________________
gen: 65 , best_of_run_f: 0.239 , best_of_run:
________________________
gen: 71 , best_of_run_f: 1.0 , best_of_run:


*********************************
END OF RUN
best_of_run attained at gen 71 and has f=1.0
init
Time:      0,  Best Fitness:       0.105721
Converged at iteration 12
Gen 0 Fitness 0
________________________
gen: 0 , best_of_run_f: 0.114 , best_of_run:
________________________
gen: 8 , best_of_run_f: 0.26 , best_of_run:
________________________
gen: 16 , best_of_run_f: 1.0 , best_of_run:


*********************************
END OF RUN
best_of_run attained at gen 16 and has f=1.0
init
Time:      0,  Best Fitness:       0.098464
Converged at iteration 25
Gen 0 Fitness 0
________________________
gen: 0 , best_of_run_f: 0.126 , best_of_run:
________________________


Time:   4000,  Best Fitness:       0.996028
Time:   4100,  Best Fitness:       0.996028
Time:   4200,  Best Fitness:       0.996028
Time:   4300,  Best Fitness:       0.996028
Time:   4400,  Best Fitness:       0.996028
Time:   4500,  Best Fitness:       0.996028
Converged at iteration 4565
Gen 0 Fitness 0
________________________
gen: 0 , best_of_run_f: 0.138 , best_of_run:
________________________
gen: 4 , best_of_run_f: 0.28 , best_of_run:
________________________
gen: 16 , best_of_run_f: 0.51 , best_of_run:
________________________
gen: 17 , best_of_run_f: 0.644 , best_of_run:
________________________
gen: 18 , best_of_run_f: 0.7 , best_of_run:
________________________
gen: 19 , best_of_run_f: 1.0 , best_of_run:


*********************************
END OF RUN
best_of_run attained at gen 19 and has f=1.0
init
Time:      0,  Best Fitness:       0.100692
Time:    100,  Best Fitness:       0.130520
Time:    200,  Best Fitness:       0.138968
Time:    300,  Best Fitness:       0.144733
T

In [734]:
# dataset = []
# with open('sample_data_file.txt','r') as f:
#     for line in f:
#         (x,y,f_xy) = map(float,line.split(' '))
#         dataset.append([x,y,f_xy])
# f.close()
dataset = generate_dataset(rastrigin,30)
FUNCTIONS = [add, sub, mul, sin, cos, pow_2]
FUNCTIONS1 = [sin, cos, pow_2] # unary operators
FUNCTIONS2 = [add, sub, mul] # binary operators
TERMINALS = ['x', 'y', 0, 1, 2, 5, 10, math.pi]
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 3    # maximal initial random tree depth
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
PROB_MUTATION   = 0.4  # per-node mutation probability 
a=0.6
GPSO(a1=1-a, a2=a/2, a3=a/2, population_size=40, time_steps=1501, dataset=dataset,func=fitness).run()

init
Time:      0,  Best Fitness:       0.522180
Time:    100,  Best Fitness:       0.570968
Time:    200,  Best Fitness:       0.589293
Time:    300,  Best Fitness:       0.589293
Time:    400,  Best Fitness:       0.655382
Time:    500,  Best Fitness:       0.655382
Time:    600,  Best Fitness:       0.655382
Time:    700,  Best Fitness:       0.655382
Time:    800,  Best Fitness:       0.655382
Time:    900,  Best Fitness:       0.655382
Time:   1000,  Best Fitness:       0.655382
Time:   1100,  Best Fitness:       0.655382
Time:   1200,  Best Fitness:       0.655382
Time:   1300,  Best Fitness:       0.662063
Time:   1400,  Best Fitness:       0.662063
Time:   1500,  Best Fitness:       0.672286


# Problem4: Symbolic Regression

In [21]:
from deap import base, gp, tools, creator, algorithms
import operator

In [308]:
dataset1 = datase
with open('dataset.txt','w') as f:
    for each in dataset1:
        f.write("%f %f %f\n"%(each[0],each[1],each[2]))

In [663]:
def protected_div(x, y):
    return x/(y+1e-5)

def exp(x):
    try:
        return math.e ** x
    except OverflowError:
        return float('inf')

def pow2(x):return x ** 2
def pow3(x):return x ** 3
def pow4(x):return x ** 4
def pow12(x):return abs(x) ** (1/2)
def pow13(x):return x ** (1/3)
def pow14(x):return abs(x) ** (1/4)
def pow_type1(x): return choice([pow2(x),pow3(x),pow4(x)])
def pow_type2(x): return choice([pow12(x),pow13(x),pow14(x)])

In [329]:
# read the dataset
dataset = []
with open('dataset.txt','r') as f:
    for line in f:
        (x,y,f_xy) = map(float,line.split(' '))
        dataset.append([x,y,f_xy])
f.close()
FUNCTIONS = [add, sub, mul, protected_div,sin, abs,pow2,pow3,pow4,pow12,pow13,pow14]
FUNCTIONS1 = [sin, cos, exp, abs,pow2,pow3,pow4,pow12,pow13,pow14]
FUNCTIONS2 = [add, sub, mul, protected_div]
TERMINALS = ['x', 'y', 1, 2, 3,4, 10, 100,math.pi]
POP_SIZE        = 120  # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 3    # maximal initial random tree depth
GENERATIONS     = 300  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 7    # size of tournament for tournament selection
XO_RATE         = 0.8  # crossover rate 
PROB_MUTATION   = 0.2  # per-node mutation probability 
runGP(dataset,1)

Gen 0
________________________
gen: 0 , best_of_run_f: 0.122 , best_of_run:
________________________
gen: 1 , best_of_run_f: 0.158 , best_of_run:
________________________
gen: 7 , best_of_run_f: 0.333 , best_of_run:
________________________
gen: 9 , best_of_run_f: 0.867 , best_of_run:
________________________
gen: 12 , best_of_run_f: 1.0 , best_of_run:


_________________________________________________
END OF RUN
best_of_run attained at gen 12 and has f=1.0
add
   pow2
      x
   pow2
      y


In [358]:
# read the dataset
dataset = []
with open('sample_data_file.txt','r') as f:
    for line in f:
        (x,y,f_xy) = map(float,line.split(' '))
        dataset.append([x,y,f_xy])
f.close()
FUNCTIONS = [add, sub, mul, protected_div,sin, abs,pow2,pow3,pow4,pow12,pow13,pow14]
FUNCTIONS1 = [sin, cos, exp, abs,pow2,pow3,pow4,pow12,pow13,pow14]
FUNCTIONS2 = [add, sub, mul, protected_div]
TERMINALS = ['x', 'y', 1, 2, 3,4, 10, 100,math.pi]
POP_SIZE        = 90  # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 4    # maximal initial random tree depth
GENERATIONS     = 1000  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 3    # size of tournament for tournament selection
XO_RATE         = 0.9  # crossover rate 
PROB_MUTATION   = 0.15  # per-node mutation probability 
runGP(dataset,7,10)

Gen 0 Fitness 0
________________________
gen: 0 , best_of_run_f: 0.532 , best_of_run:
Gen 50 Fitness 0.5852168820338727
Gen 100 Fitness 0.6090273390684754
Gen 150 Fitness 0.6147729722753792
Gen 200 Fitness 0.6147729722753792
Gen 250 Fitness 0.6147729722753792
Gen 300 Fitness 0.6147729722753792
Gen 350 Fitness 0.6147729722753792
Gen 400 Fitness 0.6147729722753792
Gen 450 Fitness 0.6147729722753792


KeyboardInterrupt: 